In [ ]:
#讀入斷詞後的內容，讀入詞彙表，建立向量空間模型，並計算相似文件
#Big Data & Business Analytics, National Taiwan University
!pip install monpa
!pip install ArticutAPI

     |████████████████████████████████| 8.5 MB 5.5 MB/s 
     |████████████████████████████████| 103 kB 5.5 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 112 kB 30.5 MB/s 
     |████████████████████████████████| 189 kB 33.6 MB/s 
     |████████████████████████████████| 52 kB 838 kB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
google-colab 1.0.0 req

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/BDBA')
os.listdir()


Mounted at /content/drive


['limit_up_term.csv',
 'limit_down_term.csv',
 'limit_down.csv',
 'limit_up.csv',
 'limit_down_ok.csv',
 'limit_up_ok.csv',
 'limit_model.npz',
 'stopwords.txt']

In [ ]:
import csv
import monpa
from monpa import utils
from sklearn.feature_extraction.text import TfidfVectorizer

def readCSV(path):
    f = open( path, newline='', encoding='utf-8') #開檔 
    rows = csv.reader(f, delimiter=',') #讀檔
    for row in rows: 
        content.append(row[0])
    f.close()

In [ ]:
def readCSV_virtual(path):
    f = open( path, newline='', encoding='utf-8') #開檔 
    rows = csv.reader(f, delimiter=',') #讀檔
    for row in rows: 
        content_virtual.append(row[1])
    f.close()
content_virtual=list()
readCSV_virtual('limit_up.csv')
readCSV_virtual('limit_down.csv')

In [ ]:
title=list()
content=list() #用串列預備儲存所有內容
termset=dict() #用字典預備儲存所有詞彙

no=0 #幫詞彙編號

readCSV('limit_up_ok.csv')
readCSV('limit_down_ok.csv')
f = open('limit_up_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

f = open('limit_down_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

In [ ]:
print(len(content)) #檢查總共有幾份內容

738


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=termset,use_idf=True) #用給定的termset建構向量，並使用idf加權

In [ ]:
X = vectorizer.fit_transform(content) #用給定的文件集做計算
X

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


<738x145 sparse matrix of type '<class 'numpy.float64'>'
	with 20348 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy import sparse
sparse.save_npz("limit_model.npz", X) #寫入整個稀疏矩陣

In [ ]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords
stopwords = stopwordslist('stopwords.txt')

In [ ]:
#給定任何一段內容當作查詢
query='倫敦鎳價又起波瀾，週四(3月24日)連續第二個交易日飆上漲停板(15%)，帶動上海鎳價週五也飆漲停板。市場傳出，有「鎳王」之稱的青山集團董事長項光達最近已回補了部分空單。\
  彭博社24日引述未具名消息人士報導，項光達本週趁鎳市暫時解封、透過倫敦金屬交易所(LME)買進鎳期貨，藉以回補空單。\
  不過，項光達及同伴僅回補數萬噸的空方部位，手上空單仍龐大。據消息，鎳市3月8日暫停交易時，項光達握有超過150,000噸空方部位，他旗下事業及交易夥伴也持有大批空單。'

#以下套用斷詞小範例

str='' #暫存本篇斷詞後的內容用
sentence_list = utils.short_sentence(query) #斷句
for item in sentence_list:
  result_cut = monpa.cut(item) #斷詞
  for term in result_cut:
    term=term.strip() #去除前後多餘空白
    if term in stopwords:
      continue
    if(len(term)>1): #若詞長>1
      str=str+' '+term
print(str)

 倫敦 波瀾 週四 3月 24日 連續 第二 交易日 飆上 漲停板 15 帶動 上海 週五 飆漲 停板 市場 傳出 鎳王 青山集團 董事長 項光達 最近 回補 部分 空單 彭博社 24日 引述 具名 消息 人士 報導 光達 鎳市 暫時 解封 透過 倫敦 金屬 交易所 LME 買進 期貨 回補 空單 項光達 同伴 回補 數萬 空方 部位 空單 龐大 消息 鎳市 3月 8日 暫停 交易 光達 握有 超過 150,000 空方 部位 旗下 事業 交易 夥伴 持有 空單


/usr/local/lib/python3.7/dist-packages/monpa/crf_layer.py:374: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [ ]:
q=vectorizer.fit_transform([str]) #將查詢也套入同一個向量空間
print(q[0:1]) #印出該向量做觀察

  (0, 136)	0.5345224838248488
  (0, 126)	0.2672612419124244
  (0, 120)	0.5345224838248488
  (0, 105)	0.2672612419124244
  (0, 77)	0.2672612419124244
  (0, 27)	0.2672612419124244
  (0, 26)	0.2672612419124244
  (0, 4)	0.2672612419124244


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(q[0:1], X).flatten() #與給定文件集的向量做相似度計算
print(cosine_similarities) #印出與每一篇的相似度做觀察

[0.04814985 0.03807994 0.03790378 0.07440138 0.02602474 0.
 0.02694865 0.02033244 0.13709086 0.05230001 0.06721601 0.03965973
 0.08523041 0.03733025 0.19401367 0.42980253 0.10984207 0.02992318
 0.06268192 0.06759337 0.04044855 0.11315795 0.06306236 0.01884917
 0.14603998 0.08685417 0.09845707 0.02903442 0.03866806 0.
 0.01894211 0.10158326 0.06370821 0.1059548  0.         0.22940141
 0.20226308 0.06493642 0.08067632 0.         0.08986901 0.
 0.13350118 0.01321749 0.08623384 0.         0.06022181 0.10156032
 0.07502703 0.02101357 0.18569329 0.10389629 0.00970872 0.01489384
 0.15823964 0.05234666 0.         0.0556219  0.03390369 0.08812846
 0.01486799 0.03962131 0.03069846 0.03738413 0.03384391 0.0855772
 0.32903665 0.08002964 0.14499128 0.11400906 0.04983962 0.03515808
 0.11164044 0.03920044 0.5948631  0.04674327 0.14610188 0.04342487
 0.         0.01361893 0.02243324 0.12994549 0.01889915 0.
 0.13452872 0.07619722 0.08965445 0.05683632 0.07243633 0.05218896
 0.03666904 0.09828239 0.042

In [ ]:
related_docs_indices = cosine_similarities.argsort() #將相似度由小至大做排序，並轉換成文件編號
#print(related_docs_indices)

In [ ]:
d=related_docs_indices[:-6:-1] #從後面取5個文件編號 (也就是相似度最大的前5名)
print(d)

[ 74  15 704 320 584]


In [ ]:
for i in d:
  print('doc ',i,' : '+content_virtual[i]) #印出這些文件內容

doc  74  : 【時報-台北電】美股四大指數昨夜整齊收高，道指再寫新頁，帶動日韓股市今早雙雙走揚。台股昨在權王貼息、台指期壓低結算、外資擴大賣超之下，失守5日線，今日跟上美股多頭腳步，早盤三大期指聯袂開高，4月台指期開盤上漲97點、為16255點，隨後大盤開高48.79點、為16264.61點。台塑(1301)配息2.4元，為六年低點，股價開低100.5元；台積電(2330)走回填息路，開高至606元；大立光(3008)開高為3355元、鴻海(2317)開高報128元，華碩(2357)跳空漲停、衝上358元，宏碁(2353)開高至28.1元、漲逾3%。在權王重返填息路，電子重兵各挾題材助攻下，大盤漲點迅速拉升至百餘點，一舉收復5日線、16300關。聯準會宣布利率維持不變且不急於提早升息，美債殖利率、恐慌指數VIX聞訊下滑，激勵美股四大指數全面收高，終場道指漲189.42點，或0.58%，收33,015.37點，再寫新紀錄；標普500指數漲12.00點，或0.30%，收3,974.71點；那指漲53.63點，或0.40%，收13,525.20點；費半指數漲37.60點，或1.22%，收3,107.92點。個股方面，蘋果跌0.65%、特斯拉漲3.68%、波音漲3.3%、開拓重工漲3.17%、美光漲3.64%；台積電ADR跌0.12%、日月光ADR漲0.80%、聯電ADR跌0.92%、中華電信ADR跌0.41%。周三台股在權王台積電貼息、台指期壓低結算之下，大盤收跌97.34點、為16215.82點，5日線告失，成交值增至3112.25億元。籌碼面，外資連3賣、並擴大賣超129.4265億元，投信連13買、買超5.072億元，自營商賣超10.7123億元，三大法人賣超135.06億元；八大公股行庫轉買超34.8億元，為3月9日以最多。分析師指出，近三日台幣表現偏弱，台股恐向下整理，不過聯準會立場較市場預期樂觀，若主力權值股轉強，台股可望重返五日線上，選股可留意高殖利率概念股波段操作。(編輯整理：李慧蘭)
doc  15  : 【時報記者莊丙農台北報導】陽明(2609)今天恢復正常交易，股價漲停開出，長榮(2603)獲美系外資調高目標價，也以上漲5.7％開出，不過，由於近期航運股頻頻爆量，籌碼仍待沈澱，盤中呈現多空交戰。海運價格狂飆，陽明獲利大幅成長，股價去年12月起

In [ ]:
len(content_virtual)

740